In [1]:
import pandas as pd
import numpy as np
#import hvplot.pandas
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('./data/names_df.csv')
df['Name'] = df.Name.str.strip('--')
df['Number'] = df.Number.str.strip('L00').astype(np.int)
df['Year'] = df.Year.str.strip('Y').astype(np.int)
idx = df[df.Year == 88].index[0]
df.at[idx, 'Year'] = 1888

In [3]:
df['Sex'] = df.Sex.apply(lambda x: 0 if x=='F' else 1)
df.drop(columns=['Year'], inplace=True)
groups = df.groupby(['Sex', 'Name'], as_index=False).sum()
group = df.groupby(['Name']).sum()
group.drop(columns='Sex', inplace=True)

In [10]:
group.loc['Sarah']

Number    979313.0
Male        3095.0
Female    976218.0
Name: Sarah, dtype: float64

In [11]:
group.loc['Sara']

Number    393734.0
Male        1048.0
Female    392686.0
Name: Sara, dtype: float64

In [4]:
male = groups[groups.Sex==1].drop(columns=['Sex'])
male.set_index('Name', inplace=True)
fem = groups[groups.Sex==0].drop(columns=['Sex'])
fem.set_index('Name', inplace=True)

In [5]:
for name, row in group.iterrows():
    total = row.Number
    try: 
        n_male = male.at[name, 'Number']
        group.loc[name, 'Male'] = int(n_male)
    except:
        group.loc[name, 'Male'] = 0
        
group['Female'] = group.Number - group.Male
group.head()

,Number,Male,Female
Name,,,
Aaban,107,107.0,0.0
Aabha,35,0.0,35.0
Aabid,10,10.0,0.0
Aabir,5,5.0,0.0
Aabriella,32,0.0,32.0


In [271]:
male_vect = CountVectorizer(analyzer='char_wb', ngram_range=(2,8))
male_counts = male_vect.fit_transform(male.index)
male_counts = male_counts.sum(axis=0).A1
m_vocab = male_vect.get_feature_names()
mdf = pd.Series(male_counts, index=m_vocab)

fem_vect = CountVectorizer(analyzer='char_wb', ngram_range=(2,8))
fem_counts = fem_vect.fit_transform(fem.index)
fem_counts = fem_counts.sum(axis=0).A1
f_vocab = fem_vect.get_feature_names()
fdf = pd.Series(fem_counts, index=f_vocab)

In [272]:
m_norm = np.sum(male_counts)
m_norm

1217237

In [269]:
mdf.head()

 a        3682
 aa        157
 aab         3
 aaba        1
 aaban       1
dtype: int64

In [257]:
%%time

d = dict()
for v in vocab:
    try:
        m = mdf.loc[v]
    except:
        m = 0
    try:
        f = fdf.loc[v]
    except:
        f = 0
        
    d[v] = [m, f]

CPU times: user 18.4 s, sys: 71.9 ms, total: 18.5 s
Wall time: 18.5 s


In [263]:
p = pd.DataFrame.from_dict(d, orient='index', columns=['Male', 'Female'])

In [274]:
p['p_male'] = p.Male / (p.Male + p.Female)
p['p_female'] = p.Female / (p.Male + p.Female)

In [275]:
p.head()

,Male,Female,p_male,p_female
a,3682,7604,0.326245,0.673755
aa,157,257,0.379227,0.620773
aab,3,2,0.600000,0.400000
aaba,1,0,1.000000,0.000000
aaban,1,0,1.000000,0.000000


In [284]:
T = len(df.Sex)
M = df.Sex.sum()
F = T - M

In [286]:
[M/T, F/T]

[0.40857603790789565, 0.5914239620921043]

In [283]:
df.Sex.sum(), (len(df.Sex) - df.Sex.sum()), len(df.Sex)

(786372, 1138293, 1924665)

In [ ]:
class_counts = defaultdict(int)

In [186]:
f_norm = fem_counts.sum(axis=0)
f_norm

2162771

array([7604,  257,    2, ...,    1,    1,    1], dtype=int64)

In [250]:
vocab = set(m_vocab) | set(f_vocab)
vocab = sorted(vocab)

In [89]:
tm = mdf.iloc[:10]
tf = fdf.iloc[:10]

In [139]:
len(m_vocab) + len(f_vocab)

759251

In [170]:
s = pd.Series(index=vocab)

 a         NaN
 aa        NaN
 aab       NaN
 aaba      NaN
 aaban     NaN
 aaban     NaN
 aabh      NaN
 aabha     NaN
 aabha     NaN
 aabi      NaN
 aabid     NaN
 aabid     NaN
 aabir     NaN
 aabir     NaN
 aabr      NaN
 aabri     NaN
 aabrie    NaN
 aabriel   NaN
 aad       NaN
 aada      NaN
 aada      NaN
 aadam     NaN
 aadam     NaN
 aadan     NaN
 aadan     NaN
 aadar     NaN
 aadars    NaN
 aadarsh   NaN
 aade      NaN
 aaden     NaN
            ..
zzmu       NaN
zzmun      NaN
zzmun      NaN
zzmy       NaN
zzmyn      NaN
zzmyn      NaN
zzmyne     NaN
zzmyne     NaN
zzmynn     NaN
zzmynn     NaN
zzo        NaN
zzo        NaN
zzon       NaN
zzon       NaN
zzt        NaN
zzt        NaN
zzu        NaN
zzur       NaN
zzure      NaN
zzure      NaN
zzy        NaN
zzy        NaN
zzyb       NaN
zzybe      NaN
zzybel     NaN
zzybell    NaN
zzybella   NaN
zzyz       NaN
zzyzx      NaN
zzyzx      NaN
Length: 637808, dtype: float64

In [93]:
tf

 a          7604
 aa          257
 aab           2
 aabh          1
 aabha         1
 aabha         1
 aabr          1
 aabri         1
 aabrie        1
 aabriel       1
dtype: int64